In [1]:
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf   
import numpy as np
import tflearn            
import random
import json

[nltk_data] Downloading package punkt to C:\Users\Manvi
[nltk_data]     Mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with open("intents.json") as json_data: 
    intents = json.load(json_data)      

In [3]:
words=[]
documents = []
classes = []


ignore = ["?"]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        w = nltk.word_tokenize(pattern) 
        
        words.extend(w) 
        
        documents.append((w, intent["tag"]))
        
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  
            #print(classes)
            

In [4]:

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words)))  

classes = sorted(list(set(classes)))

print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

106 Documents 

37 Classes 

118 Stemmed Words 


In [5]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = [] 

    pattern_words = doc[0] 
    
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [6]:
random.shuffle(training) 
training = np.array(training) 

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Manvi Mehar\AppData\Local\Temp\ipykernel_24884\451365208.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training) #Converting training data into numpy array


In [7]:
tf.compat.v1.reset_default_graph()

#Building our own Neural Network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") 

Training Step: 13999  | total loss: 0.18566 | time: 0.031s
| Adam | epoch: 1000 | loss: 0.18566 - acc: 0.9904 -- iter: 104/106
Training Step: 14000  | total loss: 0.16901 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.16901 - acc: 0.9914 -- iter: 106/106
--
INFO:tensorflow:C:\Users\Manvi Mehar\OneDrive - The University of Colorado Denver\AI-Chatbot-master\AI-Chatbot-master\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
import pickle

pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [9]:
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [10]:
import json
with open("intents.json") as json_data:
    intents = json.load(json_data)  

In [11]:
model.load("./model.tflearn") 

INFO:tensorflow:Restoring parameters from C:\Users\Manvi Mehar\OneDrive - The University of Colorado Denver\AI-Chatbot-master\AI-Chatbot-master\model.tflearn


In [12]:
def clean_up_sentence(sentence):
    
    
    sentence_words = nltk.word_tokenize(sentence) 
    
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

def bow(sentence, words, show_details=False):
    
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [13]:
context = {} 

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    results = model.predict([bow(sentence, words)])[0]
    
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    if results:
        
        while results:
            for i in intents['intents']:
                
                if i['tag'] == results[0][0]:
                    
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        return print(random.choice(i['responses']))

            results.pop(0)

In [17]:
response("What are your working hours?")

Our working hours are 9AM to 9PM every day


In [21]:
response("What services do you provide?")

We provide Web Penetration Testing,Android Penetration Testing,Docker Penetration Testing,Vulnerability Assessment,Cyber Crime investigation and many more services.


In [22]:
response("What is Penetration?")

Penetration Testing is the process of finding vulnerabilities on the target. In this case, the organization would have set up all the security measures they could think of and would want to test if there is any other way that their system/network can be hacked.


In [23]:
response("What is vulnerability assessment?")

Vulnerability Assessment is the process of finding flaws on the target. Here, the organization knows that their system/network has flaws or weaknesses and want to find these flaws and prioritize the flaws for fixing.


In [24]:
response("bye")

Bye! Come back again soon.
